In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = ['a', 'b', 'c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [3]:
seq_data = ['word', 'wood', 'deep','dive','cold','cool','load','love','kiss','kind']

In [4]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        inputs = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[inputs])
        target_batch.append(target)
    return input_batch, target_batch

In [5]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

In [6]:
X = tf.placeholder(tf.float32, [None,n_step, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [7]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [8]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [9]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [10]:
cost =  tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X:input_batch, Y:target_batch})
    print('Epoch:', '%04d'%(epoch + 1),'cost =','{:.6}'.format(loss))
print('최적화 완료')

Epoch: 0001 cost = 3.54343
Epoch: 0002 cost = 2.61812
Epoch: 0003 cost = 1.57362
Epoch: 0004 cost = 1.29197
Epoch: 0005 cost = 0.617883
Epoch: 0006 cost = 0.979807
Epoch: 0007 cost = 0.614216
Epoch: 0008 cost = 0.532453
Epoch: 0009 cost = 0.625883
Epoch: 0010 cost = 0.253952
Epoch: 0011 cost = 0.235305
Epoch: 0012 cost = 0.252957
Epoch: 0013 cost = 0.20044
Epoch: 0014 cost = 0.315085
Epoch: 0015 cost = 0.118574
Epoch: 0016 cost = 0.070621
Epoch: 0017 cost = 0.0778821
Epoch: 0018 cost = 0.105778
Epoch: 0019 cost = 0.0894452
Epoch: 0020 cost = 0.0855818
Epoch: 0021 cost = 0.174548
Epoch: 0022 cost = 0.0350364
Epoch: 0023 cost = 0.0114164
Epoch: 0024 cost = 0.0177126
Epoch: 0025 cost = 0.0196922
Epoch: 0026 cost = 0.0449284
Epoch: 0027 cost = 0.0295367
Epoch: 0028 cost = 0.00731006
Epoch: 0029 cost = 0.0484848
Epoch: 0030 cost = 0.0290237
최적화 완료


In [12]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [13]:
input_batch, target_batch = make_batch(seq_data)

In [16]:
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X:input_batch, Y:target_batch})

In [22]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    print('\n=== 예측 결과 ===')
    print('입력값:', [w[:3] + '' for w in seq_data])
    print('예측값:', predict_words)
    print('정확도:',accuracy_val)


=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep', 'dive']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep', 'dive', 'cold']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load']
정확도: 1.0

=== 예측 결과 ===
입력값: ['wor', 'woo',